In [ ]:
# Install required packages.
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

# Helper function for visualization.
%matplotlib inline
import networkx as nx
import matplotlib.pyplot as plt

2.3.0+cu121
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 78.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 59.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [1]:
import os.path as osp
import os
import torch
import torch_geometric
from torch_geometric.datasets import AMiner
from torch_geometric.nn import MetaPath2Vec

In [2]:
path = r'C:\Users\16822\Research Project SER\SEP-NHANES\lmolina3\src\utils\metapath2vec\Aminer'
dataset = AMiner(path)
data = dataset[0]

Extracting C:\Users\16822\Research Project SER\SEP-NHANES\lmolina3\src\utils\metapath2vec\Aminer\net_aminer.zip
Extracting C:\Users\16822\Research Project SER\SEP-NHANES\lmolina3\src\utils\metapath2vec\Aminer\raw\label.zip
Processing...
Done!


In [3]:
print(data)

HeteroData(
  author={
    y=[246678],
    y_index=[246678],
    num_nodes=1693531,
  },
  venue={
    y=[134],
    y_index=[134],
    num_nodes=3883,
  },
  paper={ num_nodes=3194405 },
  (paper, written_by, author)={ edge_index=[2, 9323605] },
  (author, writes, paper)={ edge_index=[2, 9323605] },
  (paper, published_in, venue)={ edge_index=[2, 3194405] },
  (venue, publishes, paper)={ edge_index=[2, 3194405] }
)


In [4]:
print(data['author'])

{'y': tensor([0, 2, 5,  ..., 0, 1, 5]), 'y_index': tensor([ 168866, 1327323,     870,  ...,  168759,  254769,  264374]), 'num_nodes': 1693531}


In [21]:
from sklearn.model_selection import train_test_split

# Extract the node indices and labels
author_labels = data['author']['y']
author_indices = data['author']['y_index']
num_author_nodes = data['author']['num_nodes']
print(num_author_nodes)
# print(author_indices)

# Calculate the number of nodes for each split
num_train = int(0.10 * num_author_nodes)
num_train_dec = num_train/num_author_nodes
num_val = int(0.10 * num_author_nodes)
num_test = num_author_nodes - num_train - num_val


# Calculate the number of nodes for each split
num_train = int(0.10 * num_author_nodes)
num_val = int(0.10 * num_author_nodes)  # Note: we'll use this only for proportional split

# First split: Train (10%) and the remaining (90%)
train_indices, remaining_indices, train_labels, remaining_labels = train_test_split(
    author_indices, author_labels, train_size=num_train_dec, stratify=author_labels, random_state=42)

# Proportion of remaining data for validation (10% of total, or 1/9 of remaining)
val_proportion = num_val / (num_author_nodes - num_train)
print(val_proportion)
# Second split: Validation (1/9 of remaining) and Test (8/9 of remaining)
val_indices, test_indices, val_labels, test_labels = train_test_split(
    remaining_indices, remaining_labels, train_size=val_proportion, stratify=remaining_labels, random_state=42)


# Initialize the masks
train_mask = torch.zeros(num_author_nodes, dtype=torch.bool)
val_mask = torch.zeros(num_author_nodes, dtype=torch.bool)
test_mask = torch.zeros(num_author_nodes, dtype=torch.bool)

# Set the appropriate mask values to True
train_mask[train_indices] = True
val_mask[val_indices] = True
test_mask[test_indices] = True

# Assign the masks to the data
data['author']['train_mask'] = train_mask
data['author']['val_mask'] = val_mask
data['author']['test_mask'] = test_mask

print("Train mask:", train_mask.sum().item())
print("Validation mask:", val_mask.sum().item())
print("Test mask:", test_mask.sum().item())


1693531
0.11111103821207234
Train mask: 24667
Validation mask: 24667
Test mask: 197344


In [ ]:


# Print the overall structure of the dataset
print("Overall structure of the dataset:")
print(data)

# List all node types and their features
print("\nNode types and their features:")
for node_type in data.node_types:
    print(f"Node type: {node_type}")
    for key, value in data[node_type].items():
        print(f"  {key}: {value}")

# List all edge types and their indices
print("\nEdge types and their indices:")
for edge_type in data.edge_types:
    print(f"Edge type: {edge_type}")
    print(f"  Edge index shape: {data[edge_type].edge_index.shape}")

# Example of accessing specific node features
print("\nExample of accessing specific node features:")
author_features = data['author']
print("Author features:", author_features)

# Example of accessing specific edge indices
print("\nExample of accessing specific edge indices:")
author_paper_edge_index = data['author', 'writes', 'paper'].edge_index
print("Author writes paper edge index:", author_paper_edge_index)


Overall structure of the dataset:
HeteroData(
  author={
    y=[246678],
    y_index=[246678],
    num_nodes=1693531,
  },
  venue={
    y=[134],
    y_index=[134],
    num_nodes=3883,
  },
  paper={ num_nodes=3194405 },
  (paper, written_by, author)={ edge_index=[2, 9323605] },
  (author, writes, paper)={ edge_index=[2, 9323605] },
  (paper, published_in, venue)={ edge_index=[2, 3194405] },
  (venue, publishes, paper)={ edge_index=[2, 3194405] }
)

Node types and their features:
Node type: author
  y: tensor([0, 2, 5,  ..., 0, 1, 5])
  y_index: tensor([ 168866, 1327323,     870,  ...,  168759,  254769,  264374])
  num_nodes: 1693531
Node type: venue
  y: tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5,
        5, 5, 5,

In [ ]:
metapath = [
    ('author', 'writes', 'paper'),
    ('paper', 'published_in', 'venue'),
    ('venue', 'publishes', 'paper'),
    ('paper', 'written_by', 'author'),
]

if torch.cuda.is_available():
    print('Cuda')
    device = torch.device('cuda')
elif torch_geometric.is_xpu_available():
    print('xpu')
    device = torch.device('xpu')
else:
    print('cpu')
    device = torch.device('cpu')

model = MetaPath2Vec(data.edge_index_dict, embedding_dim=128,
                     metapath=metapath, walk_length=50, context_size=7,
                     walks_per_node=5, num_negative_samples=5,
                     sparse=True).to(device)

loader = model.loader(batch_size=128, shuffle=True, num_workers=6)
optimizer = torch.optim.SparseAdam(list(model.parameters()), lr=0.01)



Cuda


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 6 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [ ]:

def train(epoch, log_steps=100, eval_steps=2000):
    model.train()

    total_loss = 0
    for i, (pos_rw, neg_rw) in enumerate(loader):
        optimizer.zero_grad()
        loss = model.loss(pos_rw.to(device), neg_rw.to(device))
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        if (i + 1) % log_steps == 0:
            print((f'Epoch: {epoch}, Step: {i + 1:05d}/{len(loader)}, '
                   f'Loss: {total_loss / log_steps:.4f}'))
            total_loss = 0

        if (i + 1) % eval_steps == 0:
            acc = test()
            print((f'Epoch: {epoch}, Step: {i + 1:05d}/{len(loader)}, '
                   f'Acc: {acc:.4f}'))


@torch.no_grad()
def test(train_ratio=0.1):
    model.eval()

    z = model('author', batch=data['author'].y_index.to(device))
    y = data['author'].y

    perm = torch.randperm(z.size(0))
    train_perm = perm[:int(z.size(0) * train_ratio)]
    test_perm = perm[int(z.size(0) * train_ratio):]

    return model.test(z[train_perm], y[train_perm], z[test_perm], y[test_perm],
                      max_iter=150)



In [ ]:

for epoch in range(1, 6):
    train(epoch)
    acc = test()
    print(f'Epoch: {epoch}, Accuracy: {acc:.4f}')


Epoch: 1, Step: 00100/13231, Loss: 9.1016
Epoch: 1, Step: 00200/13231, Loss: 7.5671
Epoch: 1, Step: 00300/13231, Loss: 6.4565
Epoch: 1, Step: 00400/13231, Loss: 5.8505
Epoch: 1, Step: 00500/13231, Loss: 5.5900
Epoch: 1, Step: 00600/13231, Loss: 5.4316
Epoch: 1, Step: 00700/13231, Loss: 5.2982
Epoch: 1, Step: 00800/13231, Loss: 5.1681
Epoch: 1, Step: 00900/13231, Loss: 5.0449
Epoch: 1, Step: 01000/13231, Loss: 4.9268
Epoch: 1, Step: 01100/13231, Loss: 4.8116
Epoch: 1, Step: 01200/13231, Loss: 4.7003
Epoch: 1, Step: 01300/13231, Loss: 4.5951
Epoch: 1, Step: 01400/13231, Loss: 4.4940
Epoch: 1, Step: 01500/13231, Loss: 4.3962
Epoch: 1, Step: 01600/13231, Loss: 4.2956
Epoch: 1, Step: 01700/13231, Loss: 4.2026
Epoch: 1, Step: 01800/13231, Loss: 4.1093
Epoch: 1, Step: 01900/13231, Loss: 4.0237
Epoch: 1, Step: 02000/13231, Loss: 3.9375
Epoch: 1, Step: 02000/13231, Acc: 0.2870
Epoch: 1, Step: 02100/13231, Loss: 3.8531
Epoch: 1, Step: 02200/13231, Loss: 3.7697
Epoch: 1, Step: 02300/13231, Loss: 

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 6 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Epoch: 2, Step: 00100/13231, Loss: 1.1491
Epoch: 2, Step: 00200/13231, Loss: 1.1452
Epoch: 2, Step: 00300/13231, Loss: 1.1410
Epoch: 2, Step: 00400/13231, Loss: 1.1360
Epoch: 2, Step: 00500/13231, Loss: 1.1325
Epoch: 2, Step: 00600/13231, Loss: 1.1285
Epoch: 2, Step: 00700/13231, Loss: 1.1246
Epoch: 2, Step: 00800/13231, Loss: 1.1202
Epoch: 2, Step: 00900/13231, Loss: 1.1171
Epoch: 2, Step: 01000/13231, Loss: 1.1132
Epoch: 2, Step: 01100/13231, Loss: 1.1093
Epoch: 2, Step: 01200/13231, Loss: 1.1056
Epoch: 2, Step: 01300/13231, Loss: 1.1026
Epoch: 2, Step: 01400/13231, Loss: 1.0988
Epoch: 2, Step: 01500/13231, Loss: 1.0948
Epoch: 2, Step: 01600/13231, Loss: 1.0920
Epoch: 2, Step: 01700/13231, Loss: 1.0887
Epoch: 2, Step: 01800/13231, Loss: 1.0855
Epoch: 2, Step: 01900/13231, Loss: 1.0820
Epoch: 2, Step: 02000/13231, Loss: 1.0794
Epoch: 2, Step: 02000/13231, Acc: 0.8506
Epoch: 2, Step: 02100/13231, Loss: 1.0761
Epoch: 2, Step: 02200/13231, Loss: 1.0732
Epoch: 2, Step: 02300/13231, Loss: 

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 6 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Epoch: 3, Step: 00100/13231, Loss: 0.9057
Epoch: 3, Step: 00200/13231, Loss: 0.9049
Epoch: 3, Step: 00300/13231, Loss: 0.9039
Epoch: 3, Step: 00400/13231, Loss: 0.9036
Epoch: 3, Step: 00500/13231, Loss: 0.9032
Epoch: 3, Step: 00600/13231, Loss: 0.9026
Epoch: 3, Step: 00700/13231, Loss: 0.9019
Epoch: 3, Step: 00800/13231, Loss: 0.9010
Epoch: 3, Step: 00900/13231, Loss: 0.9008
Epoch: 3, Step: 01000/13231, Loss: 0.8997
Epoch: 3, Step: 01100/13231, Loss: 0.8995
Epoch: 3, Step: 01200/13231, Loss: 0.8991
Epoch: 3, Step: 01300/13231, Loss: 0.8982
Epoch: 3, Step: 01400/13231, Loss: 0.8973
Epoch: 3, Step: 01500/13231, Loss: 0.8969
Epoch: 3, Step: 01600/13231, Loss: 0.8965
Epoch: 3, Step: 01700/13231, Loss: 0.8961
Epoch: 3, Step: 01800/13231, Loss: 0.8951
Epoch: 3, Step: 01900/13231, Loss: 0.8947
Epoch: 3, Step: 02000/13231, Loss: 0.8945
Epoch: 3, Step: 02000/13231, Acc: 0.8929
Epoch: 3, Step: 02100/13231, Loss: 0.8938
Epoch: 3, Step: 02200/13231, Loss: 0.8934
Epoch: 3, Step: 02300/13231, Loss: 

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 6 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Epoch: 4, Step: 00100/13231, Loss: 0.8581
Epoch: 4, Step: 00200/13231, Loss: 0.8577
Epoch: 4, Step: 00300/13231, Loss: 0.8571
Epoch: 4, Step: 00400/13231, Loss: 0.8573
Epoch: 4, Step: 00500/13231, Loss: 0.8569
Epoch: 4, Step: 00600/13231, Loss: 0.8568
Epoch: 4, Step: 00700/13231, Loss: 0.8564
Epoch: 4, Step: 00800/13231, Loss: 0.8566
Epoch: 4, Step: 00900/13231, Loss: 0.8564
Epoch: 4, Step: 01000/13231, Loss: 0.8560
Epoch: 4, Step: 01100/13231, Loss: 0.8555
Epoch: 4, Step: 01200/13231, Loss: 0.8559
Epoch: 4, Step: 01300/13231, Loss: 0.8553
Epoch: 4, Step: 01400/13231, Loss: 0.8551
Epoch: 4, Step: 01500/13231, Loss: 0.8551
Epoch: 4, Step: 01600/13231, Loss: 0.8546
Epoch: 4, Step: 01700/13231, Loss: 0.8548
Epoch: 4, Step: 01800/13231, Loss: 0.8545
Epoch: 4, Step: 01900/13231, Loss: 0.8544
Epoch: 4, Step: 02000/13231, Loss: 0.8543
Epoch: 4, Step: 02000/13231, Acc: 0.9063
Epoch: 4, Step: 02100/13231, Loss: 0.8540
Epoch: 4, Step: 02200/13231, Loss: 0.8540
Epoch: 4, Step: 02300/13231, Loss: 

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 6 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Epoch: 5, Step: 00100/13231, Loss: 0.8417
Epoch: 5, Step: 00200/13231, Loss: 0.8414
Epoch: 5, Step: 00300/13231, Loss: 0.8414
Epoch: 5, Step: 00400/13231, Loss: 0.8412
Epoch: 5, Step: 00500/13231, Loss: 0.8411
Epoch: 5, Step: 00600/13231, Loss: 0.8415
Epoch: 5, Step: 00700/13231, Loss: 0.8410
Epoch: 5, Step: 00800/13231, Loss: 0.8410
Epoch: 5, Step: 00900/13231, Loss: 0.8409
Epoch: 5, Step: 01000/13231, Loss: 0.8409
Epoch: 5, Step: 01100/13231, Loss: 0.8408
Epoch: 5, Step: 01200/13231, Loss: 0.8407
Epoch: 5, Step: 01300/13231, Loss: 0.8407
Epoch: 5, Step: 01400/13231, Loss: 0.8404
Epoch: 5, Step: 01500/13231, Loss: 0.8405
Epoch: 5, Step: 01600/13231, Loss: 0.8402
Epoch: 5, Step: 01700/13231, Loss: 0.8403
Epoch: 5, Step: 01800/13231, Loss: 0.8404
Epoch: 5, Step: 01900/13231, Loss: 0.8403
Epoch: 5, Step: 02000/13231, Loss: 0.8405
Epoch: 5, Step: 02000/13231, Acc: 0.9144
Epoch: 5, Step: 02100/13231, Loss: 0.8401
Epoch: 5, Step: 02200/13231, Loss: 0.8399
Epoch: 5, Step: 02300/13231, Loss: 

In [ ]:
10% training,10%validation, 80% test, splitting the training mask and split evenly by the label
DBLP hetero
LastFM
author node; venue nodes; paper nodes

run DBLP and LastFM the same splits

heterogenous graph masked autoencoders
git hub look for it. reimplement model,

1. drug traffickers is a big problem in united states and in social media
2. dealers people who follow them, freinds are high likely to be drug traffickers

2. limited dataset to train drug trafficking model. Manually labeled and annotated dataset,
3. build model to handle inbalanced data, bc pos users are much smaller than neg users.


RUn/gat/gin on hetergoenous graphs ie. aminer, dblp and lastfm.

